In [1]:
import pandas as pd
import numpy as np
import string
import nltk

from sklearn.utils import shuffle

In [2]:
fake = pd.read_csv("C:\\Users\\Praveen\\OneDrive\\Desktop\\fake news detection\\data\\raw data\\fake news detection\\Fake.csv", dtype=str)
true = pd.read_csv("C:\\Users\\Praveen\\OneDrive\\Desktop\\fake news detection\\data\\raw data\\fake news detection\\True.csv", dtype=str)
print("Fake shape (raw):", fake.shape)
print("True shape (raw):", true.shape)

Fake shape (raw): (23311, 172)
True shape (raw): (8499, 4)


In [3]:
fake = fake.loc[:, ~fake.columns.str.contains("^Unnamed")]
true = true.loc[:, ~true.columns.str.contains("^Unnamed")]

print("Fake columns after cleanup:", fake.columns.tolist())
print("True columns after cleanup:", true.columns.tolist())

Fake columns after cleanup: ['title', 'text', 'subject', 'date']
True columns after cleanup: ['title', 'text', 'subject', 'date']


In [4]:
fake["target"] = 0   # Fake news
true["target"] = 1   # True news

In [5]:
fake = fake[["title", "text", "target"]]
true = true[["title", "text", "target"]]

In [6]:
data = pd.concat([fake, true], axis=0)
data = shuffle(data, random_state=42).reset_index(drop=True)

print("Combined dataset shape:", data.shape)

Combined dataset shape: (31810, 3)


In [7]:
print("Null values:\n", data.isnull().sum())
print("Duplicates:", data.duplicated().sum())

Null values:
 title     15455
text      15455
target        0
dtype: int64
Duplicates: 15499


In [8]:
data['title'] = data['title'].fillna("")
data['text'] = data['text'].fillna("")

In [9]:
# Combine title + text
data["final_text"] = data["title"] + " " + data["text"]

In [10]:
data = data[data['final_text'].str.strip().str.len() > 10]

In [11]:
print("Before duplicate removal:", data.shape)

data = data.drop_duplicates(subset=['final_text'])
data = data.reset_index(drop=True)

print("After duplicate removal:", data.shape)

Before duplicate removal: (16355, 4)
After duplicate removal: (16310, 4)


In [12]:
data["final_text"] = data["final_text"].str.lower()

In [13]:
def remove_punctuation(text):
    return "".join([char for char in text if char not in string.punctuation])

data["final_text"] = data["final_text"].apply(remove_punctuation)

In [14]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

data["final_text"] = data["final_text"].apply(
    lambda x: " ".join(word for word in x.split() if word not in stop_words)
)

In [15]:
data["final_text"] = data["final_text"].str.replace(r"\s+", " ", regex=True).str.strip()

In [16]:
data["target"].value_counts()

target
1    8454
0    7856
Name: count, dtype: int64

In [18]:
data.to_csv("../data/preprocessed data/cleaned_fakenews.csv", index=False)
print("✅ Cleaned dataset saved as cleaned_fakenews.csv")

✅ Cleaned dataset saved as cleaned_fakenews.csv
